# Zadanie rekrutacyjne 3 - Regresja logistyczna

Twoim zadaniem jest opracowanie __modelu regresji logistycznej__.

## Analiza danych

Przeanalizuj dane dostarczone w pliku `dane_zadanie_v1.0.xlsx`. 

Czy na ich podstawie zdecydujesz się opracować model regresji logistycznej? Uzasadnij odpowiedź.

Jeśli decydujesz się opracować model napisz także:
- Czy mimo wszystko masz jakieś wątpliwości? 
- Czy brakuje Ci informacji, które mogłyby w Twojej ocenie pomóc?

In [ ]:
# analiza danych

Tutaj wpisz i uzasadnij decyzję dotyczącą opracowania modelu.

## Opracowanie modelu

Jeżeli nie zdecydujesz się opracować modelu, pozostaw tę sekcję pustą.

Jeśli decydujesz się opracować model, zamodeluj  `Target` w oparciu o zmienne `Zm1`-`Zm5` (zauważ, że `Zm3_a/b/c` są komponentami zmiennej `Zm3`). 

Możesz dokonywać standardowych przekształceń zmiennych `Zm` aby poprawić jakość działania modelu. 

Przedstaw i krótko opisz kolejne kroki modelowania.

Oceń jakość otrzymanego modelu.

In [ ]:
# tu wpisz rozwiązanie